In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
btvote = pd.read_pickle('../data/btvote.pkl')
btvote.head()

,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
2,Linke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
3,CDU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN
4,Linke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN


# NaN handling

### Delete full NaN groups
Delete all votes where all values of one party are NaN

In [3]:
unique_groups = btvote['party'].unique()
for group_value in unique_groups:
    group_df = btvote[btvote['party'] == group_value]
    nan_columns = group_df.columns[group_df.isna().all()]
    for column in nan_columns:
        print(column, end=' ')
    btvote = btvote.drop(nan_columns, axis=1)

vote_19042 vote_19047 vote_19048 vote_19051 vote_19054 vote_19060 vote_19072 vote_19080 vote_19097 vote_19106 vote_19132 vote_19151 vote_19152 vote_19162 vote_19194 vote_19207 vote_19219 vote_19061 vote_19077 vote_19095 vote_19100 vote_19103 vote_19200 vote_19221 vote_19224 vote_19228 vote_19079 vote_19085 vote_19123 vote_19143 vote_19144 vote_19211 vote_19182 

### Data Enrichment
Imputing of missing values (NaN)

- sklearn [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) with strategies *mean* and *mode/most_frequent*
- sklearn [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer) with different *n_neighbors*
- sklearn **custom** GroupImputer with strategies *mean* and *mode*:

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a custom Imputer that imputes based on the party
class GroupImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean'):
        self.strategy = strategy

    def fit(self, X, y):
        # store the y series to find the correct index in the transform function
        self.y = y
        # self.group_values becomes an attribute that is the map of group_specific values
        if self.strategy == 'mode':
            self.group_values = X.groupby(self.y).agg(lambda x: pd.Series.mode(x)[0])
        else:
            self.group_values = X.groupby(self.y).mean()
        return self

    def transform(self, X):
        for column in X.columns:
            if X[column].isna().any():
                # apply to all columns that contain any NaN values:
                # if the current cell in the DataFrame is a NaN value, return the group-specific value for that cell
                # othwerwise return the original value
                X[column] = X.apply(
                    lambda row: self.group_values.loc[self.y[row.name],column] if np.isnan(row[column])
                                   else row[column], axis=1)
        return X.to_numpy()

# Split data and encode target variable

In [5]:
# Split dataframe in 'data' and 'target'
btvote_data = btvote.drop('party', axis=1)
btvote_target = btvote['party']

# Encode the target variable
label_encoder = preprocessing.LabelEncoder()
btvote_target = label_encoder.fit_transform(btvote_target)

# Pipeline and GridSearch setup

In [22]:
from imblearn.pipeline import Pipeline
# imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
# balancing
from imblearn.over_sampling import RandomOverSampler
# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Pipeline
# 1. Imputing NaN values
# 2. Balancing
# 3. Classification
pipeline = Pipeline([('imputer', None), ('balancing', RandomOverSampler()), ('estimator', None)])

# set parameters
# TODO: extend parameter field by Classifiers. This is just an example of how the pipeline and GridSearch work on our data
parameters = [
    {
        'imputer': [SimpleImputer(strategy='mean'), SimpleImputer(strategy='most_frequent'), KNNImputer(), GroupImputer(strategy='mean'), GroupImputer(strategy='mode')],
        'estimator': [KNeighborsClassifier()],
        'estimator__n_neighbors': range(2, 8),
    }, {
        'imputer': [SimpleImputer(strategy='mean'), SimpleImputer(strategy='most_frequent'), KNNImputer(), GroupImputer(strategy='mean'), GroupImputer(strategy='mode')],
        'estimator': [DecisionTreeClassifier()],
        'estimator__max_depth': [3,6],
    }
]

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# specify the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# define the scoring function
f1 = make_scorer(f1_score, average='micro')

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
# TODO: maybe use nested cross validation in the future
grid_search_estimator.fit(btvote_data, btvote_target)

# print the results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_estimator__n_neighbors,param_imputer,param_estimator__max_depth,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010389,0.006286,0.012366,0.005631,KNeighborsClassifier(n_neighbors=2),2,SimpleImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.813333,0.746667,0.837838,0.851351,0.837838,0.864865,0.878378,0.843027,0.038879,5
1,0.029273,0.008472,0.012496,0.006735,KNeighborsClassifier(n_neighbors=2),2,SimpleImputer(strategy='most_frequent'),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.813333,0.746667,0.837838,0.851351,0.837838,0.878378,0.878378,0.843045,0.037435,4
2,0.007898,0.008207,0.010462,0.006950,KNeighborsClassifier(n_neighbors=2),2,KNNImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.853333,...,0.813333,0.760000,0.837838,0.851351,0.864865,0.864865,0.878378,0.843063,0.032239,3
3,0.029105,0.005936,0.037211,0.027410,KNeighborsClassifier(n_neighbors=2),2,GroupImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.813333,0.760000,0.851351,0.864865,0.851351,0.878378,0.878378,0.849766,0.035310,1
4,0.284424,0.013426,0.031820,0.007822,KNeighborsClassifier(n_neighbors=2),2,GroupImputer(strategy='mode'),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.813333,0.746667,0.837838,0.837838,0.837838,0.878378,0.891892,0.844378,0.039874,2
5,0.006220,0.007646,0.014733,0.005856,KNeighborsClassifier(n_neighbors=2),3,SimpleImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.786667,0.773333,0.824324,0.837838,0.810811,0.824324,0.891892,0.833586,0.035381,9
6,0.030187,0.006334,0.014787,0.005514,KNeighborsClassifier(n_neighbors=2),3,SimpleImputer(strategy='most_frequent'),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.893333,...,0.786667,0.773333,0.837838,0.837838,0.810811,0.824324,0.891892,0.837604,0.039339,7
7,0.005729,0.006867,0.011147,0.007325,KNeighborsClassifier(n_neighbors=2),3,KNNImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.866667,...,0.786667,0.760000,0.824324,0.851351,0.810811,0.824324,0.891892,0.830937,0.037667,10
8,0.026987,0.007425,0.034352,0.007775,KNeighborsClassifier(n_neighbors=2),3,GroupImputer(),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.880000,...,0.786667,0.773333,0.837838,0.824324,0.810811,0.851351,0.891892,0.837622,0.036939,6
9,0.304480,0.027366,0.028820,0.007889,KNeighborsClassifier(n_neighbors=2),3,GroupImputer(strategy='mode'),NaN,{'estimator': KNeighborsClassifier(n_neighbors...,0.880000,...,0.786667,0.773333,0.824324,0.851351,0.810811,0.851351,0.905405,0.836324,0.039761,8
